In [39]:
from dotenv import load_dotenv
from httpx import AsyncClient
from langchain.chat_models import ChatOpenAI

load_dotenv()
rq = AsyncClient()
chat_model = ChatOpenAI(http_client=rq, model="gpt-3.5-turbo-1106", temperature=0.6)

In [40]:
from langchain.schema import HumanMessage

text = "Hello, how are you?"
messages = [HumanMessage(content=text)]


res = await chat_model.ainvoke(input=messages)  # type: ignore
res

AIMessage(content="Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?")

### Load documents from URLs

In [41]:
# fixes a bug with asyncio and jupyter
import nest_asyncio
from langchain.document_loaders.sitemap import SitemapLoader

nest_asyncio.apply()

reference_url = "https://uh.edu/recreation/facilities/facility-hours"

sitemap_loader = SitemapLoader(web_path=reference_url)
sitemap_loader.requests_kwargs = {"verify": False}

docs = sitemap_loader.aload()
docs[0]

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.87it/s]


Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFacility Hours - University of Houston\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content Restrict my search to Campus Recreation Submit Close\n\n\n\n\nStudent Affairs Login to AccessUHGive to UH SearchCampus RecreationHomeHoursAbout UsMission, Vision & ValuesHealth and WellbeingCoogsCARETelling Our StoryHistorySpotlightsRenovation ProjectsData and StatisticsArtworkCampus Recreation StaffStudent EmploymentNews & AnnouncementsAquaticsAdult Swim LessonsPrivate Swim LessonsSafety TrainingSpecial EventsReservations and RentalsAquatic PoliciesRenovation UpdatesFacilities and RentalsFacility HoursFacility Renovations 23-24Live Facility CountsFacility PoliciesFacility ToursReservations and Special EventsFacility DetailsRenovation and UpgradesCampus Recreation - On the GoFitnessGroup FitnessSmall Group Training Personal TrainingFitness AssessmentsFIT for HIRECertification TrainingNutrition ConsultationSpecial EventsActive with ArtRED 

In [42]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.qdrant import Qdrant

embedding = OpenAIEmbeddings()

vector_store = await Qdrant.afrom_documents(
    docs, embedding=embedding, location=":memory:", collection_name=reference_url
)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever

VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f2bb8365910>, search_kwargs={'k': 1})

In [43]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Here is today's date and time with timezone info:
{datetime}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [44]:
from datetime import datetime
from operator import itemgetter

from dateutil import tz
from langchain.memory import ConversationBufferMemory
from langchain.schema import format_document
from langchain_core.messages import get_buffer_string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
)

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

time_str_format = "%A %B %d, %Y %I:%M %p %Z"
tzinfo = tz.gettz("America / Chicago")

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "datetime": lambda x: datetime.now(tz=tzinfo).strftime(time_str_format),
    "question": itemgetter("question"),
}

# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | chat_model,
    "docs": itemgetter("docs"),
}

final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [45]:
async def query_facility_hours(question: str):
    inputs = {"question": question}
    return await final_chain.ainvoke(inputs)

In [52]:
await query_facility_hours("what time ranges is the natatorium open on monday?")

{'answer': AIMessage(content='The opening hours of the natatorium on Monday are 6:00 am - 9:00 am and 12 - 9:00 pm.'),
 'docs': [Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFacility Hours - University of Houston\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content Restrict my search to Campus Recreation Submit Close\n\n\n\n\nStudent Affairs Login to AccessUHGive to UH SearchCampus RecreationHomeHoursAbout UsMission, Vision & ValuesHealth and WellbeingCoogsCARETelling Our StoryHistorySpotlightsRenovation ProjectsData and StatisticsArtworkCampus Recreation StaffStudent EmploymentNews & AnnouncementsAquaticsAdult Swim LessonsPrivate Swim LessonsSafety TrainingSpecial EventsReservations and RentalsAquatic PoliciesRenovation UpdatesFacilities and RentalsFacility HoursFacility Renovations 23-24Live Facility CountsFacility PoliciesFacility ToursReservations and Special EventsFacility DetailsRenovation and UpgradesCampus Recreation - On the GoFitnessGroup FitnessSmall Group Train